## Start by setting week number
#### Wait for ESPN data update until at least Wednesday morning, but run latest by Thursday night

https://www.espn.com/nfl/stats/team/_/stat/total/season/2021/seasontype/2

In [1]:
SET_WEEK = 6
week_name = 'week_' + str(SET_WEEK)
print(week_name)

week_6


In [2]:
from scrapers.scrape_2021 import get_2021_data
import pandas as pd
import numpy as np
import pickle

import warnings
warnings.filterwarnings('ignore')

### Models

In [3]:
rf = '../modeling/rf.pkl'
gbm = '../modeling/gbm.pkl'
lgb = '../modeling/lgb.pkl'
catboost = '../modeling/catboost.pkl'

with open(rf, 'rb') as file:
    RF = pickle.load(file)
    
with open(gbm, 'rb') as file:
    GBM = pickle.load(file)
    
with open(lgb, 'rb') as file:
    LGB = pickle.load(file)
    
with open(catboost, 'rb') as file:
    CATBOOST= pickle.load(file)

### 2021 data

In [4]:
%%time

X = get_2021_data()

# from boruta elimination method:
keep_cols = ['offense_points_per_game', 'offense_passing_AVG', 'offense_passing_YDS/G', 'offense_passing_RTG', 
             'offense_receiving_AVG', 'offense_rushing_YDS/G', 'defense_points_per_game', 'defense_passing_AVG', 
             'defense_receiving_AVG', 'defense_rushing_YDS/G', 'defense_passing_SYL_per_game', 
             'offense_downs_Third Downs_ATT_per_game', 'defense_downs_Third Downs_ATT_per_game', 
             'defense_downs_Fourth Downs_ATT_per_game', 'defense_pass_to_rush_ratio', 
             'defense_downs_First Downs_penalty_ratio', 'offense_passing_TD_per_game', 'offense_pass_TD_to_INT']

X = X[['Team', 'games_played', 'season'] + keep_cols].copy()

X.head(2)

Wall time: 16.5 s


,Team,games_played,season,offense_points_per_game,offense_passing_AVG,offense_passing_YDS/G,offense_passing_RTG,offense_receiving_AVG,offense_rushing_YDS/G,defense_points_per_game,...,defense_receiving_AVG,defense_rushing_YDS/G,defense_passing_SYL_per_game,offense_downs_Third Downs_ATT_per_game,defense_downs_Third Downs_ATT_per_game,defense_downs_Fourth Downs_ATT_per_game,defense_pass_to_rush_ratio,defense_downs_First Downs_penalty_ratio,offense_passing_TD_per_game,offense_pass_TD_to_INT
0,Dallas Cowboys,6,2021,34.2,8.5,296.5,115.3,11.5,164.3,24.3,...,12.4,86.2,16.166667,12.500000,11.666667,1.500000,1.892562,0.090909,2.666667,4.00
1,Kansas City Chiefs,6,2021,30.8,7.8,308.5,103.1,11.3,125.0,29.3,...,12.7,133.2,10.166667,11.333333,11.000000,1.333333,1.322581,0.063830,3.000000,2.25


In [5]:
X.fillna(0, inplace = True)
X.replace([np.inf, -np.inf], 0, inplace=True)

In [8]:
X['games_played'].unique().tolist()

[6, 5]

In [9]:
predictions = X[['Team', 'season']].copy()
predictions['week_marker'] = week_name

predictions['pred_rf'] = RF.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_gbm'] = GBM.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_lgb'] = LGB.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_catboost'] = CATBOOST.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_mean'] = (predictions['pred_rf'] + predictions['pred_gbm'] + predictions['pred_lgb'] + predictions['pred_catboost']) / 4

predictions.sort_values('pred_mean', ascending = False, inplace = True)
predictions.head(3)

,Team,season,week_marker,pred_rf,pred_gbm,pred_lgb,pred_catboost,pred_mean
6,Arizona Cardinals,2021,week_6,0.310732,0.347820,0.397527,0.341286,0.349341
5,Buffalo Bills,2021,week_6,0.285716,0.246341,0.411262,0.381045,0.331091
7,Los Angeles Rams,2021,week_6,0.320543,0.321711,0.341944,0.335369,0.329892


In [10]:
predictions.to_csv('weekly_runs/' + week_name + '.csv', index = False)

### Check weekly run

In [11]:
predictions

,Team,season,week_marker,pred_rf,pred_gbm,pred_lgb,pred_catboost,pred_mean
6,Arizona Cardinals,2021,week_6,0.310732,0.347820,0.397527,0.341286,0.349341
5,Buffalo Bills,2021,week_6,0.285716,0.246341,0.411262,0.381045,0.331091
7,Los Angeles Rams,2021,week_6,0.320543,0.321711,0.341944,0.335369,0.329892
0,Dallas Cowboys,2021,week_6,0.273340,0.230925,0.331979,0.286301,0.280636
2,Tampa Bay Buccaneers,2021,week_6,0.258340,0.241237,0.281447,0.296365,0.269347
20,Cincinnati Bengals,2021,week_6,0.190307,0.149405,0.425622,0.265483,0.257704
1,Kansas City Chiefs,2021,week_6,0.230701,0.190478,0.228799,0.237651,0.221907
9,Las Vegas Raiders,2021,week_6,0.213193,0.151662,0.241602,0.270377,0.219208
8,Cleveland Browns,2021,week_6,0.162502,0.096461,0.166832,0.221778,0.161893
3,Baltimore Ravens,2021,week_6,0.202255,0.116946,0.111725,0.197609,0.157134
